## BERT Embedding 计算
本程序使用 BERT 计算文本的词向量，并输出为 NumPy 的 `.npy` 文件。

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
import json

### 配置参数
设定输入文件、BERT 预训练模型、批处理大小等参数。

In [ ]:
tokenized_jsonl = ""  # 分词后得到的JSONL
model_name = "bert-base-chinese"  # 预训练模型
output_file = ""           # 嵌入向量保存路径
batch_size = 16                   # 批大小

### 读取 JSONL, 获取文本
本示例将 tokens 拼接成空格分隔的字符串后送入 BERT，也可使用原始句子。

In [ ]:
sentences = []
with open(tokenized_jsonl, 'r', encoding='utf-8') as fin:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        token_list = record.get("tokens", [])
        text = " ".join(token_list)
        sentences.append(text)

print(f"文本条数: {len(sentences)}")
if sentences:
    print(f"预览第一条: {sentences[0]}")

### 加载 BERT 预训练模型和 Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

### 设置计算设备
自动检测是否有 GPU 可用。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # 设置为评估模式

### 切分批次
通过 DataLoader 处理数据，以便批量输入。

In [ ]:
data_loader = DataLoader(sentences, batch_size=batch_size, shuffle=False)

### 计算 BERT Embedding 并提取 CLS 向量

In [ ]:
cls_embeddings = []

for batch_sentences in tqdm(data_loader):
    inputs = tokenizer(
        batch_sentences,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    
    cls_batch = outputs.last_hidden_state[:, 0].cpu().numpy()
    cls_embeddings.append(cls_batch)

### 拼接所有批次的 CLS 向量

In [ ]:
cls_embeddings_np = np.vstack(cls_embeddings)
print("最终生成的词向量", type(cls_embeddings_np), cls_embeddings_np.shape)

### 保存到 `.npy` 文件

In [ ]:
np.save(output_file, cls_embeddings_np)
print(f"词向量存储于: {output_file}")

### 测试加载 `.npy` 文件

In [ ]:
embeddings = np.load(output_file)
print("加载回来，验证一下：", type(embeddings), embeddings.shape)